In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Users\\nisha\\Downloads\\chatbot\\intents.json').read()
intents = json.loads(data_file)

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('C:\\Users\\nisha\\Downloads\\chatbot\\words.pkl','wb'))
pickle.dump(classes,open('C:\\Users\\nisha\\Downloads\\chatbot\\classes.pkl','wb'))

38 documents
12 classes ['email_management', 'goodbye', 'greeting', 'joke', 'meeting_scheduler', 'note_retrieval', 'options', 'personal_information', 'python_is_best', 'task_management', 'task_reminder', 'thanks']
85 unique lemmatized words ["'s", ',', '.', ':', '[', ']', 'a', 'add', 'am', 'an', 'and', 'any', 'anyone', 'are', 'at', 'awesome', 'be', 'beginner', 'best', 'bye', 'can', 'chatting', 'could', 'date', 'day', 'do', 'email', 'event', 'ever', 'for', 'fun', 'funny', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'joke', 'language', 'later', 'laugh', 'learn', 'make', 'me', 'meeting', 'my', 'need', 'new', 'next', 'nice', 'note', 'offered', 'on', 'programming', 'provide', 'python', 'remind', 'retrieve', 'saved', 'see', 'some', 'start', 'support', 'task', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'what', 'when', 'where', 'which', 'you']


In [4]:
import numpy as np

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features
random.shuffle(training)

# Convert both bag and output_row lists to NumPy arrays
bag_array = np.array([bag for bag, _ in training])
output_row_array = np.array([output_row for _, output_row in training])

# Create the final training array by concatenating bag and output_row arrays along axis=1
training = np.concatenate((bag_array, output_row_array), axis=1)

# Split the training data into X (patterns) and Y (intents)
train_x = training[:, :-len(classes)]
train_y = training[:, -len(classes):]

print("Training data created")


Training data created


In [5]:
from keras.optimizers import SGD

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains number of neurons
# equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)



Epoch 1/200
8/8 [==============================] - 1s 4ms/step - loss: 2.4718 - accuracy: 0.1053
Epoch 2/200
8/8 [==============================] - 0s 3ms/step - loss: 2.4088 - accuracy: 0.2105
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 2.3498 - accuracy: 0.1579
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 2.1973 - accuracy: 0.2632
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 2.1798 - accuracy: 0.3684
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9493 - accuracy: 0.4737
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9431 - accuracy: 0.2895
Epoch 8/200
8/8 [==============================] - 0s 3ms/step - loss: 1.7939 - accuracy: 0.4737
Epoch 9/200
8/8 [==============================] - 0s 3ms/step - loss: 1.5828 - accuracy: 0.5789
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.4957 - accuracy: 0.6579
Epoch 11/200
8/8 [===========

C:\Users\nisha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('C:\\Users\\nisha\\Downloads\\chatbot\\intents.json').read())
words = pickle.load(open('C:\\Users\\nisha\\Downloads\\chatbot\\words.pkl','rb'))
classes = pickle.load(open('C:\\Users\\nisha\\Downloads\\chatbot\\classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
    

In [8]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Personal Assistant ChatBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


1/1 [==============================] - 0s 31ms/step
